# 用于读取MAIA云产品文件的示例脚本。

使用以下工具创建：
包 | 版本--- | ---
SatPy | 0.9.0PyResample | 1.10.1Trollimage | 1.5.3PyKdtree | 1.3.1

MAIA文件可以使用AAPP以及MAIA4_RUN和MAIA4_RUN_AVHRR脚本创建。
针对 BOTH VIIRS an AVHRR 成像仪
- 安装 satpy 和 pyresample- 如果您想生成带有海岸线的图表，也可以安装 pycoast- 以下是一个用于读取MAIA文件的最简脚本示例
MAIA 文件使用特定的命名语法，例如：
- avhCT_M02_GL_20151012_S005503_E005802_DES_D_La-11_Lo0126_00000.h5    
- viiCT_npp_DB_20121010_S132824_E132947_ASC_D_La050_Lo-012_00001.h5    
参考文献 :
https://nwpsaf.eu/site/software/aapp/

[NWPSAF-MF-UD-003] 数据格式      
[NWPSAF-MF-UD-009] MAIA 第4版科学用户手册      
本示例使用了MAIA云产品，该产品可在随AAPP软件提供的MAIA4_test.tgz tar测试用例文件中找到。

In [1]:
from satpy import Scene
from satpy.utils import debug_on
import numpy as np
from satpy.composites import PaletteCompositor
from satpy.writers import to_image

In [2]:
# define palette (matplotlib style)
cpool = ['#4e7791',  # not processed 0
         '#008c30',  # Cloud free land 1
         '#000000',  # Cloud free sea 2
         '#ffbbff',  # Snow over ice 3
         '#dda0dd',  # Sea ice 4
         '#ffa500',  # Very low cumuliforme 5      # non utilisé par maia
         '#ff6600',  # Very low no cumuliforme 6
         '#ffd800',  # Low cumuliforme 7           # non utilisé par maia
         '#ffa500',  # Low cumuliforme 8
         '#ffff00',  # Mid-level cumuliforme 9         # non utilisé par maia
         '#d8ff00',  # Mid-level no cumuliforme 10
         '#cccc00',  # High opaque cumuliforme 11    # on utilisé par maia
         '#d8b575',  # High opaque no cumuliforme 12
         '#ffffff',  # Very High opaque cumuliforme 13 # non utilisé par maia
         '#ffe0aa',  # Very High opaque no cumuliforme 14
         '#0000ff',  # Semi transp. thin 15
         "#00b2ff",  # Semi transp. meanly thick 16
         '#00ffe5',  # Semi transp. thick 17
         '#00ffb2',  # semi transp abobe others 18
         '#d800ff',  # fractionnal
         '#660f00',  # Not classified 20
         ]


In [3]:
# function to convert it in satpy style
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return [int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)]


In [4]:
fnmaia = "/home/a000680/data/maia/viiCT_npp_DB_20121010_S132824_E132947_ASC_D_La050_Lo-012_00001.h5"

In [5]:
maia_scene = Scene(sensor='viirs', reader='maia', filenames=[fnmaia])

/home/a000680/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
maia_scene.all_dataset_names()

['Alt_surface',
 'CM_qual',
 'CloudMask',
 'CloudTopPres',
 'CloudTopTemp',
 'CloudType',
 'Latitude',
 'Longitude',
 'Mask_in',
 'Mask_ind',
 'Sat_zenith',
 'Tsurf',
 'cma',
 'cma_conf',
 'ct',
 'fov_qual',
 'land_water_background',
 'opaq_cloud']

In [7]:
maia_scene.load(["CloudType", "ct", "cma", "cma_conf",
                     'opaq_cloud', "CloudTopPres", "CloudTopTemp", "Alt_surface"])


CloudType 是一个位字段，包含实际的 'ct'，其值范围从 0 到 20可以根据cpool颜色映射进行解释
“ct”可以以黑白显示：

In [8]:
maia_scene.show("ct")
maia_scene.save_dataset("ct", filename="ct_bw.png")

<img src="ct_bw.png">

但最好对图像进行调色板处理：
步骤 1: 创建调色板

In [9]:
mycolors = []
for i in range(21):
    mycolors.append(hex_to_rgb(cpool[i]))
arr = np.array(mycolors)
np.save("/tmp/binary_maia_ct_colormap.npy", arr)

步骤2：复合物的创建

In [10]:
compositor = PaletteCompositor("test", standard_name="maia_ct")
composite = compositor((maia_scene["ct"], arr))
img = to_image(composite)
img.show()
img.pil_save("ct.png")

<img src="ct.png">
